# XGBoost Model Predictions



In [1]:
# Setup Environment

import os
import logging
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import xgboost as xgb
import optuna
import itertools
from sklearn.preprocessing import LabelEncoder
import pyspark.sql.functions as F
from pyspark.sql.functions import (col, count, row_number, abs, unix_timestamp, mean, 
                                   when, lit, min as F_min, max as F_max , upper, trim,
                                   row_number, mean as F_mean, countDistinct, last, first, when)
from src.data_preprocessing.data_prep1.data_utils import initialize_environment 
# Set global references to None
spark = None
master_results_df = None
df = None
training_data = None
train_df = None

In [3]:

spark, jdbc_url, jdbc_properties, parquet_dir, log_file = initialize_environment()


Spark session created successfully.


In [4]:
upcoming_races = spark.read.parquet("/home/exx/myCode/horse-racing/FoxRiverAIRacing/data/parquet/upcoming_races.parquet")
training_data = spark.read.parquet("/home/exx/myCode/horse-racing/FoxRiverAIRacing/data/parquet/train_df")

In [5]:
training_data.count()

394322

In [6]:
upcoming_races.count()

1078

# Switching to Pandas

In [7]:
# Convert Spark DataFrame -> Pandas DataFrame
upcoming_races = upcoming_races.toPandas()
training_data = training_data.toPandas()
# Quick info about the DataFrame
#print(df.info())
#print(df.head(5))

## Set the race_id

In [8]:
upcoming_races["race_id"] = (
    upcoming_races["course_cd"].astype(str) + "_" +
    upcoming_races["race_date"].astype(str) + "_" +
    upcoming_races["race_number"].astype(str)
)

## Group and sort data by race_id and group_id

In [9]:
# Generate unique numeric group_id from race_id
upcoming_races["group_id"] = upcoming_races["race_id"].astype("category").cat.codes

In [10]:
# Sort by race_id for consistency
upcoming_races = upcoming_races.sort_values("group_id", ascending=True)
upcoming_races.reset_index(drop=True, inplace=True)

## Drop Non-numeric Features

In [11]:
unused_columns = [
    # columns you do NOT use in features or group_id
    "race_date", "date_of_birth"
]
cols_to_drop = [col for col in unused_columns if col in upcoming_races.columns]

upcoming_races.drop(columns=cols_to_drop, inplace=True)
print("After dropping unused cols, shape:", upcoming_races.shape)

After dropping unused cols, shape: (1078, 77)


# Convert DataTime columns to Numerical Values

In [12]:
# Convert datetime columns to numerical
upcoming_races["first_race_date_5"] = pd.to_datetime(upcoming_races["first_race_date_5"])
upcoming_races["most_recent_race_5"] = pd.to_datetime(upcoming_races["most_recent_race_5"])
upcoming_races["prev_race_date"] = pd.to_datetime(upcoming_races["prev_race_date"])

# Calculate numeric date features
upcoming_races["first_race_date_5_numeric"] = (upcoming_races["first_race_date_5"] - pd.Timestamp("1970-01-01")).dt.days
upcoming_races["most_recent_race_5_numeric"] = (upcoming_races["most_recent_race_5"] - pd.Timestamp("1970-01-01")).dt.days
upcoming_races["prev_race_date_numeric"] = (upcoming_races["prev_race_date"] - pd.Timestamp("1970-01-01")).dt.days

# Drop original datetime columns
upcoming_races.drop(columns=["first_race_date_5", "most_recent_race_5", "prev_race_date"], inplace=True)


## Set Rank/Label - Calculate Frequency Encoding from Historical Data:

Use historical data to calculate the frequency of each horse_id.

In [13]:
# If the largest official_fin is 20 (some races can have 20 horses),
# then label = (21 - official_fin).
# So official_fin=1 => label=20, official_fin=2 =>19, etc.
# If your max is 14, you can do (15 - official_fin).  Just ensure "best" horse has largest label.
if "official_fin" in training_data.columns:
    # Calculate 'rank' and add it to the DataFrame
    training_data["rank"] = 21 - training_data["official_fin"]
    # Drop the 'official_fin' column
    training_data.drop(columns=["official_fin"], inplace=True)

## Simple Target Encoding for (XGBoost/LightGBM)

In [14]:
# Ensure original horse_id is kept
upcoming_races["horse_id_original"] = upcoming_races["horse_id"]

def encode_horse_id_with_training_mean(upcoming_df, training_df, horse_col, target_col):
    """
    Encode horse_id in the upcoming data using the historical mean target
    from the training data.

    Parameters:
    - upcoming_df: DataFrame for races to predict
    - training_df: DataFrame with historical data
    - horse_col: Column containing horse IDs
    - target_col: Target column (e.g., rank, performance metric)

    Returns:
    - DataFrame with encoded horse_id
    """
    # Compute historical mean for each horse_id from the training data
    horse_means = training_df.groupby(horse_col)[target_col].mean()

    # Map the historical means to the upcoming races
    upcoming_df["horse_id_encoded"] = upcoming_df[horse_col].map(horse_means)

    # If a horse_id in upcoming races doesn't exist in training data, fill with the global mean
    global_mean = training_df[target_col].mean()
    upcoming_df["horse_id_encoded"] = upcoming_df["horse_id_encoded"].fillna(global_mean)

    return upcoming_df

# Example usage
upcoming_races = encode_horse_id_with_training_mean(
    upcoming_df=upcoming_races,
    training_df=training_data,  # Historical data
    horse_col="horse_id",
    target_col="rank"  # Column used for encoding (from historical data)
)

# Drop the original horse_id if it's not needed
upcoming_races.drop(columns=["horse_id"], inplace=True)

# Assigned Numerical Features

In [15]:

features = ['race_number','horse_id_encoded','purse','weight','claimprice','power','morn_odds','avgspd','class_rating',
    'net_sentiment','avg_spd_sd','ave_cl_sd','hi_spd_sd','pstyerl','all_starts','all_win','all_place',
    'all_show','all_fourth','all_earnings','cond_starts','cond_win','cond_place','cond_show','cond_fourth',
    'cond_earnings','avg_speed_5','best_speed','avg_beaten_len_5','avg_dist_bk_gate1_5','avg_dist_bk_gate2_5',
    'avg_dist_bk_gate3_5','avg_dist_bk_gate4_5','avg_speed_fullrace_5','avg_stride_length_5','avg_strfreq_q1_5',
    'avg_strfreq_q2_5','avg_strfreq_q3_5','avg_strfreq_q4_5','prev_speed','speed_improvement','days_off',
    'avg_workout_rank_3','jock_win_percent','jock_itm_percent','trainer_win_percent','trainer_itm_percent',
    'jt_win_percent','jt_itm_percent','jock_win_track','jock_itm_track','trainer_win_track','trainer_itm_track',
    'jt_win_track','jt_itm_track','age_at_race_day','distance_meters', 'count_workouts_3']

In [17]:
# upcoming_races.columns

# Set the Category Columns with Label Encoder

In [51]:
cat_cols = [ "course_cd", "sex", "equip", "surface", "med",  
            "race_type", "stk_clm_md", "turf_mud_mark", "layoff_cat"]
# Convert categorical columns if needed
for col in cat_cols:
    if col in X_predict.columns:
        upcoming_races[col] = upcoming_races[col].astype("category")

# Specify categorical feature indices
cat_cols_indices = [training_data.columns.get_loc(col) for col in cat_cols]

In [52]:
# upcoming_races.columns
# # Assuming X_predict is your DataFrame

# # Set the display option to show all columns
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_info_columns', 100)

# # Display the data types of all columns
# print(training_data.dtypes)

# # Alternatively, use the info method to get a detailed summary
upcoming_races.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1078 entries, 0 to 1077
Data columns (total 78 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   course_cd                   1078 non-null   category
 1   race_number                 1078 non-null   float64 
 2   saddle_cloth_number         1078 non-null   object  
 3   horse_name                  1078 non-null   object  
 4   purse                       1078 non-null   float64 
 5   weight                      1078 non-null   float64 
 6   sex                         1078 non-null   category
 7   equip                       1078 non-null   category
 8   claimprice                  1078 non-null   float64 
 9   surface                     1078 non-null   category
 10  power                       1078 non-null   float64 
 11  med                         1078 non-null   category
 12  morn_odds                   1078 non-null   float64 
 13  avgspd            

# Make DMatrix Object

In [54]:
# Assuming upcoming_races is your DataFrame for prediction

# Create a DataFrame with only the necessary columns
X_predict = upcoming_races[features].copy()

# # Verify that the categorical columns are included in X_predict
# print("Columns in X_predict:", X_predict.columns)

# # Check the data types to ensure categorical columns are correctly converted
# print("Data types in X_predict:")
# print(X_predict.dtypes)


In [55]:
# Assuming X_predict is your DataFrame

# # Set the display option to show all columns
# pd.set_option('display.max_columns', 100)
# pd.set_option('display.max_info_columns', 100)

# # Display the data types of all columns
# print(X_predict.dtypes)

# # Alternatively, use the info method to get a detailed summary
# X_predict.info(verbose=True)

In [57]:
# Handle missing values (adjust fill strategy as needed)
# X_predict.fillna(0, inplace=True)

# Create DMatrix for prediction
dpredict = xgb.DMatrix(X_predict, feature_names=features, enable_categorical=True)

# Define the path to the saved model
model_path = "/home/exx/myCode/horse-racing/FoxRiverAIRacing/src/models/XGB_914054_optuna_ranking_model-2025-01-11.json"

# Load the model
xgb_model = xgb.Booster()
xgb_model.load_model(model_path)

# Print the model to verify it has been loaded
print(xgb_model)

# Step 3: Predict scores
predicted_scores = xgb_model.predict(dpredict)


XGBoostError: [14:44:58] /home/conda/feedstock_root/build_artifacts/xgboost-split_1733179782501/work/src/learner.cc:1462: Check failed: learner_model_param_.num_feature >= p_fmat->Info().num_col_ (58 vs. 67) : Number of columns does not match number of features in booster.
Stack trace:
  [bt] (0) /home/exx/anaconda3/envs/mamba_env/envs/rapids-23.08/lib/libxgboost.so(dmlc::LogMessageFatal::~LogMessageFatal()+0x71) [0x7f4a474c07d1]
  [bt] (1) /home/exx/anaconda3/envs/mamba_env/envs/rapids-23.08/lib/libxgboost.so(xgboost::LearnerImpl::ValidateDMatrix(xgboost::DMatrix*, bool) const+0x1e9) [0x7f4a4785bfc9]
  [bt] (2) /home/exx/anaconda3/envs/mamba_env/envs/rapids-23.08/lib/libxgboost.so(xgboost::LearnerImpl::Predict(std::shared_ptr<xgboost::DMatrix>, bool, xgboost::HostDeviceVector<float>*, int, int, bool, bool, bool, bool, bool)+0x27a) [0x7f4a4786e0fa]
  [bt] (3) /home/exx/anaconda3/envs/mamba_env/envs/rapids-23.08/lib/libxgboost.so(XGBoosterPredictFromDMatrix+0x2df) [0x7f4a473b980f]
  [bt

# Load the XGB Model

In [ ]:
# Define the path to the saved model
model_path = "/home/exx/myCode/horse-racing/FoxRiverAIRacing/src/models/XGB_914054_optuna_ranking_model-2025-01-11.json"

# Load the model
xgb_model = xgb.Booster()
xgb_model.load_model(model_path)

# Print the model to verify it has been loaded
print(xgb_model)

In [ ]:


# Step 3: Predict scores
X_predict_values = X_predict.values
predicted_scores = xgb_model.predict(X_predict_values)